In [8]:
import os
import time 
import datetime

def print_table(data: dict[str, list[float]], precision: int = 4):
    # Convert to list of keys and find number of rows
    headers = list(data.keys())
    headers = ["index"] + headers        
    n_rows = len(next(iter(data.values())))
    data["index"] = list(range(1, n_rows+1))

    # Format floats to desired precision
    fmt = f"{{:.{precision}f}}"

    # Compute column widths
    col_widths = []
    for h in headers:
        values = data[h]
        max_val_len = max(len(fmt.format(v)) for v in values)
        col_widths.append(max(len(h), max_val_len))

    # Build header row
    header_row = " | ".join(f"{h:>{w}}" for h, w in zip(headers, col_widths))
    separator = "-+-".join("-" * w for w in col_widths)

    # Print table
    print(header_row)
    print(separator)
    for i in range(n_rows):
        row = " | ".join(fmt.format(data[h][i]).rjust(w) for h, w in zip(headers, col_widths))
        print(row)

In [9]:
def dt_to_intkey(date_str: str, time_str: str) -> float:
    dt = datetime.datetime.strptime(f"{date_str} {time_str}", "%Y_%m_%d %H_%M_%S")
    return dt.timestamp()

In [37]:
data_dir = "../../logs_cluster/logs/15_12_2025_ycocg_arm_smol_no_colreg_gain_test_multiimarm_kodak/results/"
res_files_unfiltered = [f for f in os.listdir(data_dir) if f.endswith(".log")]
res_files_unfiltered.sort()
# print(*res_files_unfiltered, sep="\n")
dates, times = zip(*(s.split("__")[:2] for s in res_files_unfiltered))
creation_times = {
    # key is date_time string converted to int/float for sorting
    index: dt_to_intkey(date, time)
    for index, (date, time) in enumerate(zip(dates, times))
}
only_after = dt_to_intkey("2025_10_30", "00_00_00")
res_files = [
    res_files_unfiltered[index]
    for index, creation_time in creation_times.items()
    if creation_time >= only_after
]
# print(*res_files, sep="\n")
print(len(res_files), "files found after filtering.")

24 files found after filtering.


In [39]:
supported_colorspaces = ["RGB", "YCoCg"]
results = {
}
using_image_arm = False

for f in res_files:
    with open(os.path.join(data_dir, f), "r") as infile:
        lines = infile.readlines()
        adding_to_colorspace = None
        for line in lines:
            for key in supported_colorspaces:
                if "Using color space " + key in line:
                    adding_to_colorspace = key
                    if key not in results:
                        results[key] = [[{}] for _ in range(len(res_files_unfiltered))]
                if "Using image ARM: True" in line:
                    using_image_arm = True
        if adding_to_colorspace is None:
            print(f"Could not determine color space for file {f}, skipping.")
            continue

        for line in lines:
            if "Final results after quantization" in line:
                parts = line[len("Final results after quantization:") :].strip().split(", ")
                im_index = int(f.split("_")[-1][len("kodim") :][: -len(".log")]) - 1
                for part in parts:
                    key, value = part.split(": ")
                    results[adding_to_colorspace][im_index][-1][key] = float(value)

def list_dict_to_dict_list(
    lst: list[dict[str, float]],
) -> dict[str, list[float]]:
    if not lst:
        return {}
    keys = lst[0].keys()
    dict_list = {key: [] for key in keys}
    for d in lst:
        for key in keys:
            dict_list[key].append(d.get(key, 0.0))
    return dict_list


for key in results.keys():
    results_for_key = [d[0] for d in results[key]]
    results_for_key = list_dict_to_dict_list(results_for_key)

    print(f"{key}, Using Image ARM: {using_image_arm} Results:")
    # I screwed the value for results_for_key['Rate NN'] and results_for_key['Loss'] so now I have to fix them
    # first from every item in results_for_key['Loss'] I should subtract results_for_key['Rate NN'], then divide results_for_key['Rate NN'] by (512*768*3)
    # and then add results_for_key['Rate NN'] back to results_for_key['Loss']
    for i in range(len(results_for_key['Loss'])):
        results_for_key['Loss'][i] = results_for_key['Loss'][i] - results_for_key['Rate NN'][i]
        results_for_key['Rate NN'][i] = results_for_key['Rate NN'][i] #/ (512 * 768 * 3)
        results_for_key['Loss'][i] = results_for_key['Loss'][i] + results_for_key['Rate NN'][i]
    print_table(results_for_key)
    for key, values in results_for_key.items():
        avg = sum(values) / len(values) if values else 0
        print(f"{key}: {avg:.4f}")

YCoCg, Using Image ARM: True Results:
  index |   Loss | Rate NN | Rate Latent | Rate Img
--------+--------+---------+-------------+---------
 1.0000 | 3.2711 |  0.0934 |      0.2742 |   2.9036
 2.0000 | 3.0914 |  0.0929 |      0.4713 |   2.5272
 3.0000 | 2.5518 |  0.0923 |      0.0932 |   2.3663
 4.0000 | 2.9182 |  0.0926 |      0.1954 |   2.6302
 5.0000 | 3.4342 |  0.0932 |      0.2954 |   3.0455
 6.0000 | 3.0983 |  0.0933 |      0.1914 |   2.8136
 7.0000 | 2.7402 |  0.0927 |      0.1545 |   2.4930
 8.0000 | 3.6788 |  0.0931 |      0.1469 |   3.4388
 9.0000 | 2.8759 |  0.0931 |      0.2154 |   2.5674
10.0000 | 2.9008 |  0.0925 |      0.1211 |   2.6872
11.0000 | 3.0390 |  0.0919 |      0.0540 |   2.8931
12.0000 | 2.7902 |  0.0934 |      0.2362 |   2.4606
13.0000 | 3.6784 |  0.0930 |      0.3470 |   3.2384
14.0000 | 3.2673 |  0.0928 |      0.1397 |   3.0349
15.0000 | 2.8833 |  0.0922 |      0.0484 |   2.7427
16.0000 | 2.7894 |  0.0933 |      0.0654 |   2.6307
17.0000 | 2.8770 |  0.0927